# Dataiku Census Project

Goal: Identify the characteristics of individuals earning more or less than $50K.

### Machine Learning Pipeline Script

## Functionalities:
1. **Data Loading and Preprocessing**:
   - Handles missing values and duplicate records.
   - Encodes categorical variables using label encoding.
   - Scales numerical features for consistency.

2. **Exploratory Data Analysis (EDA)**:
   - Visualizes data distributions and relationships (e.g., income vs. gender, education, race).
   - Identifies outliers and trends in the dataset.

3. **Feature Engineering**:
   - Creates new features (e.g., `capital_net_gain`).
   - Selects the most relevant features using statistical methods (e.g., chi-square test).

4. **Class Imbalance Handling**:
   - Applies **SMOTE (Synthetic Minority Oversampling Technique)** to balance classes in the training dataset.

5. **Model Training and Evaluation**:
   - Implements and evaluates the following machine learning models:
     - **Random Forest**
     - **Logistic Regression**
     - **XGBoost**
   - Produces classification reports, confusion matrices, and precision-recall curves.

6. **Model Comparison and Hyperparameter Tuning**:
   - Compares model performance using **AUC (Area Under the Curve)** scores.
   - Optimizes the XGBoost model using a randomized grid search.

7. **Visualization**:
   - Generates and saves:
     - Feature importance plots.
     - Model comparison plots.
     - Validation and test performance metrics.

## Dependencies:
- `pandas`: For data manipulation and analysis.
- `numpy`: For numerical computations.
- `matplotlib`: For static visualizations.
- `seaborn`: For enhanced data visualizations.
- `scikit-learn`: For machine learning models and evaluation metrics.
- `imbalanced-learn`: For handling class imbalance with SMOTE.
- `xgboost`: For gradient boosting model implementation.


## Import Packages

In [5]:
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical computations
import matplotlib.pyplot as plt  # For data visualization
import seaborn as sns  # For statistical data visualization
from sklearn.model_selection import train_test_split, RandomizedSearchCV  # For data splitting and hyperparameter tuning
from sklearn.ensemble import RandomForestClassifier  # For Random Forest classification
from sklearn.linear_model import LogisticRegression  # For Logistic Regression
from xgboost import XGBClassifier  # For XGBoost classification
from sklearn.preprocessing import MinMaxScaler, LabelEncoder  # For scaling and encoding data
from sklearn.metrics import (  # For model evaluation metrics
    classification_report,
    roc_auc_score,
    confusion_matrix,
    precision_recall_curve,
)
from sklearn.feature_selection import SelectKBest, chi2  # For feature selection
from imblearn.over_sampling import SMOTE  # For handling class imbalance through oversampling

## Load & Clean Data

In [7]:
def load_and_clean_data(train_path, test_path):
    """
    Load and preprocess the training and test datasets.
    """
    # Define column names for the datasets based on provided metadata
    column_names = [
        'age', 'class_of_worker', 'detailed_industry_recode', 'detailed_occupation_recode', 'education', 
        'wage_per_hour', 'enroll_in_edu_inst_last_wk', 'marital_stat', 'major_industry_code', 
        'major_occupation_code', 'race', 'hispanic_origin', 'sex', 'member_of_a_labor_union', 'reason_for_unemployment', 
        'full_or_part_time_employment_stat', 'capital_gains', 'capital_losses', 'dividends_from_stocks', 
        'tax_filer_stat', 'region_of_previous_residence', 'state_of_previous_residence', 
        'detailed_household_and_family_stat', 'detailed_household_summary_in_household', 'instance_weight', 
        'migration_code_change_in_msa', 'migration_code_change_in_reg', 'migration_code_move_within_reg', 
        'live_in_this_house_1_year_ago', 'migration_prev_res_in_sunbelt', 'num_persons_worked_for_employer', 
        'family_members_under_18', 'country_of_birth_father', 'country_of_birth_mother', 'country_of_birth_self', 
        'citizenship', 'own_business_or_self_employed', 'fill_inc_questionnaire_for_veterans_admin', 
        'veterans_benefits', 'weeks_worked_in_year', 'year', 'income'
    ]
    
    # Load the training dataset
    train_df = pd.read_csv(train_path, header=None)  # Load CSV without a header
    train_df.columns = column_names  # Assign column names to the DataFrame
    train_df.replace('?', np.nan, inplace=True)  # Replace "?" with NaN
    train_df.dropna(inplace=True)  # Drop rows with missing values
    train_df.drop_duplicates(inplace=True)  # Remove duplicate rows
    if 'instance_weight' in train_df.columns:  # Drop unnecessary 'instance_weight' column if it exists
        train_df.drop('instance_weight', axis=1, inplace=True)

    # Split the training data into training and validation sets
    X = train_df.drop('income', axis=1)  # Features
    y = train_df['income']  # Target variable
    X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y  # Stratified split to maintain class proportions
    )

    # Load the test dataset
    test_df = pd.read_csv(test_path, header=None)  # Load CSV without a header
    test_df.columns = column_names  # Assign column names
    test_df.replace('?', np.nan, inplace=True)  # Replace "?" with NaN
    test_df.dropna(inplace=True)  # Drop rows with missing values
    test_df.drop_duplicates(inplace=True)  # Remove duplicate rows
    if 'instance_weight' in test_df.columns:  # Drop unnecessary 'instance_weight' column if it exists
        test_df.drop('instance_weight', axis=1, inplace=True)

    # Return the cleaned training dataset, splits for training and validation, and the cleaned test dataset
    return train_df, (X_train_split, X_val_split, y_train_split, y_val_split), test_df

### Notes

- **Initial Observations**:
  - The dataset exhibits a high class imbalance, with the majority of instances labeled as `-50,000`.
  - A significant number of duplicate entries exist in both the learning and testing datasets.
    

- **Assumptions and Prior Knowledge**:
  - Higher educational attainment is generally associated with higher salaries.
  - Individuals in their 30s to 40s are more likely to have higher incomes (peak income years).
  - There are racial disparities in income levels (a bias could be present in model).
  - There are gender disparities in income levels (a bias could be present in model).

## Process Data

In [10]:
def preprocess_data(train_df, splits, test_df):
    """
    Preprocess the data by performing feature engineering, label encoding, 
    and target variable transformation.
    """
    # Unpack splits
    X_train_split, X_val_split, y_train_split, y_val_split = splits

    # Feature engineering: Calculate net capital gain and drop redundant columns
    for dataset in [train_df, X_train_split, X_val_split]:
        dataset['capital_net_gain'] = dataset['capital_gains'] - dataset['capital_losses']  # Calculate net capital gain
        dataset.drop(columns=['capital_gains', 'capital_losses'], inplace=True)  # Drop original columns

    # Apply the same transformation to the test dataset
    test_df['capital_net_gain'] = test_df['capital_gains'] - test_df['capital_losses']  # Calculate net capital gain
    test_df.drop(columns=['capital_gains', 'capital_losses'], inplace=True)  # Drop original columns

    # Initialize a dictionary to store LabelEncoders for each categorical column
    label_encoders = {}

    # Encode categorical features using LabelEncoder
    for column in train_df.select_dtypes(include=['object']).columns:
        if column != 'income':  # Skip the target variable
            le = LabelEncoder()  # Initialize LabelEncoder
            train_df[column] = le.fit_transform(train_df[column])  # Fit and transform training data
            X_train_split[column] = le.transform(X_train_split[column])  # Transform training split
            X_val_split[column] = le.transform(X_val_split[column])  # Transform validation split
            test_df[column] = le.transform(test_df[column])  # Transform test data
            label_encoders[column] = le  # Store the encoder

    # Convert the target variable (`income`) to binary
    train_df['income'] = train_df['income'].apply(lambda x: 1 if x.strip() == '50000+.' else 0)
    test_df['income'] = test_df['income'].apply(lambda x: 1 if x.strip() == '50000+.' else 0)
    y_train_split = y_train_split.apply(lambda x: 1 if x.strip() == '50000+.' else 0)
    y_val_split = y_val_split.apply(lambda x: 1 if x.strip() == '50000+.' else 0)

    # Return the preprocessed datasets and the label encoders
    return train_df, (X_train_split, X_val_split, y_train_split, y_val_split), test_df, label_encoders

## Data Analysis 

In [12]:
def perform_eda(train_df):
    """
    Perform Exploratory Data Analysis (EDA) on the training dataset.
    This function generates various visualizations and saves them as image files 
    to analyze the distribution and relationships of key features in the dataset.
    """
    # Basic statistics summary
    print("Basic Statistics of Train Data:")
    print(train_df.describe())

    # Plot: Distribution of Income
    print("\nDistribution of Income:")
    sns.countplot(data=train_df, x='income', palette="viridis")
    plt.title('Income Distribution')
    plt.xlabel('Income (0 = Less than 50K, 1 = 50K+)')
    plt.ylabel('Count')
    plt.tight_layout()  # Ensure labels are not cut off
    plt.savefig('eda_income_distribution.png')
    plt.close()

    # Plot: Income vs Gender
    print("\nIncome vs Gender:")
    sns.countplot(data=train_df, x='sex', hue='income', palette="viridis")
    plt.title('Income by Gender')
    plt.xlabel('Gender (0 = Female, 1 = Male)')
    plt.ylabel('Count')
    plt.legend(title='Income', loc='upper right', labels=['Less than 50K', '50K+'])
    plt.tight_layout()
    plt.savefig('eda_income_by_gender.png')
    plt.close()

    # Plot: Income vs Education
    print("\nIncome vs Education:")
    education_mapping = {
        0: "Children", 1: "7th and 8th grade", 2: "9th grade", 3: "10th grade",
        4: "High school graduate", 5: "11th grade", 6: "12th grade no diploma",
        7: "5th or 6th grade", 8: "Less than 1st grade", 9: "Bachelors degree",
        10: "1st to 4th grade", 11: "Some college but no degree",
        12: "Masters degree", 13: "Associates degree-occup/vocational",
        14: "Associates degree-academic program", 15: "Doctorate degree",
        16: "Prof school degree"
    }
    train_df['education_label'] = train_df['education'].map(education_mapping)
    sns.countplot(data=train_df, y='education_label', hue='income', palette="viridis",
                  order=train_df['education_label'].value_counts().index)
    plt.title('Income by Education Level')
    plt.xlabel('Count')
    plt.ylabel('Education Level')
    plt.legend(title='Income', loc='upper right', labels=['Less than 50K', '50K+'])
    plt.tight_layout()
    plt.savefig('eda_income_by_education.png')
    plt.close()

    # Plot: Income vs Race
    print("\nIncome vs Race:")
    race_mapping = {
        0: "White", 1: "Black", 2: "Asian or Pacific Islander",
        3: "Amer Indian Aleut or Eskimo", 4: "Other"
    }
    train_df['race_label'] = train_df['race'].map(race_mapping)
    sns.countplot(data=train_df, y='race_label', hue='income', palette="viridis",
                  order=train_df['race_label'].value_counts().index)
    plt.title('Income by Race')
    plt.xlabel('Count')
    plt.ylabel('Race')
    plt.legend(title='Income', loc='upper right', labels=['Less than 50K', '50K+'])
    plt.tight_layout()
    plt.savefig('eda_income_by_race.png')
    plt.close()

    # Plot: Age Distribution vs Income
    print("\nAge Distribution vs Income:")
    sns.histplot(data=train_df, x='age', hue='income', bins=20, kde=True, palette="viridis", multiple="stack")
    plt.title('Age Distribution by Income')
    plt.xlabel('Age')
    plt.ylabel('Frequency')
    plt.legend(title='Income', loc='upper right', labels=['Less than 50K', '50K+'])
    plt.tight_layout()
    plt.savefig('eda_age_distribution.png')
    plt.close()

    # Plot: Boxplots for Wage, Weeks Worked, and Capital Net Gain
    print("\nBoxplots for Wage, Weeks Worked, and Capital Net Gain:")
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    sns.boxplot(data=train_df, x='income', y='wage_per_hour', ax=axes[0], hue='income', palette="viridis")
    axes[0].set_title('Wage per Hour by Income')
    axes[0].set_xlabel('Income (0 = Less than 50K, 1 = 50K+)')
    axes[0].set_ylabel('Wage per Hour')
    axes[0].legend_.remove()

    sns.boxplot(data=train_df, x='income', y='weeks_worked_in_year', ax=axes[1], hue='income', palette="viridis")
    axes[1].set_title('Weeks Worked by Income')
    axes[1].set_xlabel('Income (0 = Less than 50K, 1 = 50K+)')
    axes[1].set_ylabel('Weeks Worked')
    axes[1].legend_.remove()

    sns.boxplot(data=train_df, x='income', y='capital_net_gain', ax=axes[2], hue='income', palette="viridis")
    axes[2].set_title('Capital Net Gain by Income')
    axes[2].set_xlabel('Income (0 = Less than 50K, 1 = 50K+)')
    axes[2].set_ylabel('Capital Net Gain')
    axes[2].legend_.remove()

    plt.tight_layout()
    plt.savefig('eda_boxplots.png')
    plt.close()

    # Remove the temporary education label column
    train_df.drop(columns=['education_label'], inplace=True)

### Notes

- **Observations**:
  - Men are more likely to have higher incomes.
  - Bachelor's, Associate's, and Master's degree holders tend to have higher incomes.
  - Outliers observed in `wage_per_hour`.
  - Outliers observed in `capital_net_gain`.

## Feature Engineering 

In [15]:
def feature_engineering(splits, test_df, k=10):
    """
    Perform feature engineering on training, validation, and test datasets.
    This function scales the data, selects the top K features using the Chi-square test, 
    and retains only the selected features for model training and evaluation.
    """
    X_train_split, X_val_split, y_train_split, y_val_split = splits

    # Scale data (fit scaler only on training data)
    scaler = MinMaxScaler()
    X_train_split_scaled = pd.DataFrame(scaler.fit_transform(X_train_split), columns=X_train_split.columns)
    X_val_split_scaled = pd.DataFrame(scaler.transform(X_val_split), columns=X_val_split.columns)
    X_test_scaled = pd.DataFrame(scaler.transform(test_df.drop(columns=['income'])), columns=test_df.drop(columns=['income']).columns)

    # Select K best features (fit selector only on training data)
    selector = SelectKBest(chi2, k=k)
    X_train_split_selected = selector.fit_transform(X_train_split_scaled, y_train_split)
    X_val_split_selected = selector.transform(X_val_split_scaled)
    X_test_selected = selector.transform(X_test_scaled)

    # Retain only selected feature names
    selected_features = X_train_split.columns[selector.get_support()]
    X_train_split_selected = pd.DataFrame(X_train_split_selected, columns=selected_features)
    X_val_split_selected = pd.DataFrame(X_val_split_selected, columns=selected_features)
    X_test_selected = pd.DataFrame(X_test_selected, columns=selected_features)

    return (X_train_split_selected, X_val_split_selected, y_train_split, y_val_split), X_test_selected, selected_features

## Imbalance & Tune

In [17]:
def handle_imbalance(X_train_split, y_train_split):
    """
    Balances the training data using SMOTE (Synthetic Minority Oversampling Technique) to handle class imbalance.
    """
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_split, y_train_split)
    return X_resampled, y_resampled

def train_and_evaluate_models(X_train_split, X_val_split, y_train_split, y_val_split, selected_features):
    """
    Trains and evaluates three machine learning models (Random Forest, Logistic Regression, XGBoost) on the training and validation datasets. Outputs performance metrics and feature importance plots.
    """
    results = {}

    rf_model = RandomForestClassifier(random_state=42, n_estimators=100, class_weight="balanced")
    print("Random Forest:")
    rf_auc = evaluate_on_validation(rf_model, X_train_split, X_val_split, y_train_split, y_val_split)
    plot_feature_importance(rf_model, selected_features, model_name="Random Forest")
    results['Random Forest'] = {'AUC': rf_auc}

    lr_model = LogisticRegression(max_iter=1000, random_state=42, class_weight="balanced")
    print("Logistic Regression:")
    lr_auc = evaluate_on_validation(lr_model, X_train_split, X_val_split, y_train_split, y_val_split)
    results['Logistic Regression'] = {'AUC': lr_auc}

    scale_pos_weight = (len(y_train_split) - sum(y_train_split)) / sum(y_train_split)
    print(f"Calculated scale_pos_weight for XGBoost: {scale_pos_weight:.2f}")

    xgb_model = XGBClassifier(random_state=42, scale_pos_weight=scale_pos_weight, eval_metric='logloss')
    print("XGBoost:")
    xgb_auc = evaluate_on_validation(xgb_model, X_train_split, X_val_split, y_train_split, y_val_split)
    plot_feature_importance(xgb_model, selected_features, model_name="XGBoost")
    results['XGBoost'] = {'AUC': xgb_auc}

    return results

def hyperparameter_tuning(X_train_split, y_train_split, X_val_split, y_val_split):
    """
    Performs hyperparameter tuning on the XGBoost model using RandomizedSearchCV to optimize its parameters for better performance.
    """
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.7, 0.8, 0.9]
    }

    xgb_model = XGBClassifier(random_state=42, eval_metric='logloss')

    xgb_random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=10,
        cv=3,
        scoring='roc_auc',
        random_state=42,
        n_jobs=-1
    )

    xgb_random_search.fit(X_train_split, y_train_split)

    best_params = xgb_random_search.best_params_
    print("Best Parameters for XGBoost:", best_params)

    xgb_optimized = XGBClassifier(**best_params, random_state=42, eval_metric='logloss')
    xgb_optimized.fit(X_train_split, y_train_split)

    return xgb_optimized

## Evaluate Models

In [19]:
def evaluate_on_validation(model, X_train_split, X_val_split, y_train_split, y_val_split):
    """
    Evaluates the performance of a trained model on the validation set.
    Outputs a classification report and AUC score.
    """
    model.fit(X_train_split, y_train_split)
    predictions = model.predict(X_val_split)
    probabilities = model.predict_proba(X_val_split)[:, 1]

    print("\nValidation Set Evaluation:")
    print("Classification Report:")
    print(classification_report(y_val_split, predictions))
    print(f"AUC Score: {roc_auc_score(y_val_split, probabilities):.4f}")

    # Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y_val_split, probabilities)
    plt.plot(recall, precision, label=f'{type(model).__name__}')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve (Validation)')
    plt.legend()
    filename_prc = f"precision_recall_curve_validation_{type(model).__name__.lower()}.png"
    plt.savefig(filename_prc)
    print(f"Precision-Recall Curve saved as {filename_prc}")
    plt.close()

    # Confusion Matrix
    cm = confusion_matrix(y_val_split, predictions)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title("Confusion Matrix (Validation)")
    plt.ylabel("Actual")
    plt.xlabel("Predicted")
    filename_cm = f"confusion_matrix_validation_{type(model).__name__.lower()}.png"
    plt.savefig(filename_cm)
    print(f"Confusion Matrix saved as {filename_cm}")
    plt.close()

    return roc_auc_score(y_val_split, probabilities)

def evaluate_on_test(model, X_test, y_test):
    """
    Evaluates the final model's performance on the test set.
    Outputs a classification report and AUC score.
    """
    predictions = model.predict(X_test)
    probabilities = model.predict_proba(X_test)[:, 1]

    print("\nTest Set Evaluation:")
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print(f"AUC Score: {roc_auc_score(y_test, probabilities):.4f}")

    # Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y_test, probabilities)
    plt.plot(recall, precision, label=f'{type(model).__name__}')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve (Test)')
    plt.legend()
    filename_prc = f"precision_recall_curve_test_{type(model).__name__.lower()}.png"
    plt.savefig(filename_prc)
    print(f"Precision-Recall Curve saved as {filename_prc}")
    plt.close()

    # Confusion Matrix
    cm = confusion_matrix(y_test, predictions)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title("Confusion Matrix (Test)")
    plt.ylabel("Actual")
    plt.xlabel("Predicted")
    filename_cm = f"confusion_matrix_test_{type(model).__name__.lower()}.png"
    plt.savefig(filename_cm)
    print(f"Confusion Matrix saved as {filename_cm}")
    plt.close()

    return roc_auc_score(y_test, probabilities)

## Top Features 

In [21]:
def plot_feature_importance(model, features, model_name="Model"):
    """
    Visualizes the importance of features for models that support the feature_importances_ attribute (e.g., Random Forest, XGBoost).
    Saves the feature importance plot to a file.
    
    Args:
        model: The machine learning model.
        features (list): A list of feature names.
        model_name (str): The name of the model (used for saving the plot).
    """
    if hasattr(model, "feature_importances_"):
        importances = pd.DataFrame({
            'Feature': features,
            'Importance': model.feature_importances_
        }).sort_values(by='Importance', ascending=False)

        plt.figure(figsize=(10, 6))
        sns.barplot(data=importances, x='Importance', y='Feature', dodge=False)
        plt.title(f"Feature Importance - {model_name}")
        plt.xlabel("Feature Importance Score")
        plt.ylabel("Features")
        
        # Adjust layout to prevent labels from getting cut off
        plt.tight_layout()
        
        # Save the plot
        filename = f"feature_importance_{model_name.replace(' ', '_').lower()}.png"
        plt.savefig(filename, bbox_inches='tight')
        print(f"Feature importance plot saved as {filename}")
        plt.close()
    else:
        print(f"{model_name} does not support feature importance.")

## Model Comparison 

In [23]:
def model_comparison(results):
    """
    Compares the performance of machine learning models based on their AUC scores.
    Saves the comparison plot to a file.
    
    Args:
        results (dict): A dictionary containing model names as keys and a dictionary of metrics (e.g., AUC) as values.
    """
    print("\n### Model Comparison Recap ###")
    model_names = list(results.keys())
    metrics = ['AUC']

    data = []
    for model, metrics_dict in results.items():
        row = [model] + [metrics_dict.get(metric, "N/A") for metric in metrics]
        data.append(row)

    comparison_df = pd.DataFrame(data, columns=['Model'] + metrics)
    print(comparison_df)

    # Plot Model AUC Comparison
    plt.figure(figsize=(10, 6))
    sns.barplot(x="AUC", y="Model", data=comparison_df, palette="viridis")
    plt.title("Model AUC Comparison")
    plt.xlabel("AUC Score")
    plt.ylabel("Model")
    
    # Adjust layout to avoid labels getting cut off
    plt.tight_layout()
    
    # Save the plot
    filename = "model_auc_comparison.png"
    plt.savefig(filename, bbox_inches='tight')
    print(f"Model AUC Comparison saved as {filename}")
    plt.close()

## Main 

In [25]:
def main():
    train_path = '/Users/angieguerrero/Desktop/Dataiku Project/census_income_learn.csv'
    test_path = '/Users/angieguerrero/Desktop/Dataiku Project/census_income_test.csv'

    # Load and clean data
    train_df, splits, test_df = load_and_clean_data(train_path, test_path)

    # Preprocessing
    train_df, splits, test_df, encoders = preprocess_data(train_df, splits, test_df)

    # Perform initial EDA
    perform_eda(train_df)

    # Feature engineering
    k_features = 10
    splits, X_test_selected, selected_features = feature_engineering(splits, test_df, k=k_features)

    # Handle class imbalance on training data
    X_train_split, X_val_split, y_train_split, y_val_split = splits
    X_train_split, y_train_split = handle_imbalance(X_train_split, y_train_split)

    # Train and evaluate models on validation set
    results = train_and_evaluate_models(X_train_split, X_val_split, y_train_split, y_val_split, selected_features)

    # Compare models
    model_comparison(results)

    # Hyperparameter tuning for the best model (XGBoost in this case)
    xgb_optimized = hyperparameter_tuning(X_train_split, y_train_split, X_val_split, y_val_split)

    # Evaluate the optimized model on the test set
    y_test = test_df['income']
    test_auc = evaluate_on_test(xgb_optimized, X_test_selected, y_test)
    print(f"Final Test AUC: {test_auc:.4f}")

if __name__ == "__main__":
    main()

Basic Statistics of Train Data:
                 age  class_of_worker  detailed_industry_recode  \
count  196294.000000    196294.000000             196294.000000   
mean       34.929468         3.493286                 15.603187   
std        22.210001         1.112499                 18.106401   
min         0.000000         0.000000                  0.000000   
25%        16.000000         3.000000                  0.000000   
50%        34.000000         3.000000                  1.000000   
75%        50.000000         4.000000                 33.000000   
max        90.000000         8.000000                 51.000000   

       detailed_occupation_recode      education  wage_per_hour  \
count               196294.000000  196294.000000  196294.000000   
mean                    11.490468      10.039339      56.336505   
std                     14.498128       4.151615     277.054333   
min                      0.000000       0.000000       0.000000   
25%                      0.00

/var/folders/ff/_3kz3q7542g8wy3grh7rl0kc0000gp/T/ipykernel_29580/1124886381.py:13: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=train_df, x='income', palette="viridis")



Income vs Gender:

Income vs Education:

Income vs Race:

Age Distribution vs Income:

Boxplots for Wage, Weeks Worked, and Capital Net Gain:
Random Forest:

Validation Set Evaluation:
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.94      0.95     36783
           1       0.34      0.50      0.41      2476

    accuracy                           0.91     39259
   macro avg       0.66      0.72      0.68     39259
weighted avg       0.93      0.91      0.92     39259

AUC Score: 0.8762
Precision-Recall Curve saved as precision_recall_curve_validation_randomforestclassifier.png
Confusion Matrix saved as confusion_matrix_validation_randomforestclassifier.png
Feature importance plot saved as feature_importance_random_forest.png
Logistic Regression:

Validation Set Evaluation:
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.75      0.85     36783
           1       0.

/var/folders/ff/_3kz3q7542g8wy3grh7rl0kc0000gp/T/ipykernel_29580/597264213.py:23: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x="AUC", y="Model", data=comparison_df, palette="viridis")


Best Parameters for XGBoost: {'subsample': 0.8, 'n_estimators': 300, 'max_depth': 5, 'learning_rate': 0.2}

Test Set Evaluation:
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.92      0.95     92693
           1       0.34      0.62      0.44      6186

    accuracy                           0.90     98879
   macro avg       0.66      0.77      0.69     98879
weighted avg       0.93      0.90      0.91     98879

AUC Score: 0.9032
Precision-Recall Curve saved as precision_recall_curve_test_xgbclassifier.png
Confusion Matrix saved as confusion_matrix_test_xgbclassifier.png
Final Test AUC: 0.9032


### Summary of Results

#### **Basic Statistics of Train Data**
- **Demographics**:
  - **Mean Age**: 34.93 years
  - **Education Mean**: 10.04 (Indicating at least a high school education level on average)
- **Economic Indicators**:
  - **Wage per Hour**: Mean = $56.34, Max = $9,999 (Significant outliers identified)
  - **Weeks Worked in Year**: Mean = 23.55 weeks, Max = 52 weeks (Indicating part-time employment for most individuals)
  - **Capital Net Gain**: Mean = $403.94, Max = $99,999 (Substantial outliers detected)
- **Income Distribution**:
  - **Low Income (<$50K)**: 93.69% of individuals
  - **High Income (≥$50K)**: Only 6.31% of individuals, highlighting class imbalance.

#### **Key Insights from Exploratory Data Analysis**
- **Income Distribution**:
  - A stark imbalance exists between low-income and high-income groups.
- **Income by Gender**:
  - Men are significantly more likely to belong to the high-income group than women.
- **Income by Education**:
  - Advanced degrees (Bachelor's, Associate's, and Master's) are highly correlated with higher incomes.
- **Income by Race**:
  - White individuals form the largest group of high-income earners, whereas minority races have a much lower representation in this category.
- **Age Distribution and Income**:
  - Individuals aged 30–50 are most likely to earn high incomes.
- **Outliers**:
  - Extreme values are evident in `wage_per_hour` and `capital_net_gain`, requiring careful consideration during modeling.

#### **Model Performance on Validation Set**
1. **Random Forest**:
   - **AUC**: 0.876
   - **F1-Score (High Income)**: 0.41
   - **Precision (High Income)**: 34%
   - **Recall (High Income)**: 50%
2. **Logistic Regression**:
   - **AUC**: 0.891
   - **F1-Score (High Income)**: 0.31
   - **Precision (High Income)**: 19%
   - **Recall (High Income)**: 88%
3. **XGBoost**:
   - **AUC**: 0.901
   - **F1-Score (High Income)**: 0.43
   - **Precision (High Income)**: 32%
   - **Recall (High Income)**: 67%

#### **Final Test Evaluation (XGBoost)**
- **AUC**: 0.903 (Highest performance among all models)
- **Precision (High Income)**: 34% (Moderate accuracy in identifying high-income individuals)
- **Recall (High Income)**: 62% (Captures a significant portion of the high-income group)
- **Accuracy**: 90% (High overall classification accuracy)

#### **Conclusion**
- **XGBoost** is the best-performing model, demonstrating the highest AUC (0.903) and an effective balance between precision and recall.